In [1]:
from arcgis.gis import GIS
import sgccat20lib
import arcpy
import requests
import json

In [2]:
gis = GIS(profile='sgc20')
# gis = GIS("pro")
# gis = GIS("https://editorcatastral2dev.esri.co/portal", "ejecutor1", "Esrico2021$")
profile_storaged = sgccat20lib.profile_storaged

In [3]:
dictparams = sgccat20lib.ObtenerParametros()
parcel_service = dictparams["PARCEL_SERVICE_NAME"]

In [ ]:
search_result = gis.content.search('{}'.format(parcel_service),'Feature Layer')

In [ ]:
for table in search_result[0].tables:
    print(table.properties.name)

In [ ]:
# Step: Devolver flujo de ejecutor
class ToolDevolucionFlujo(object):
    """Devolver flujo de ejecutor"""
    PARENT = "PARENT"
    WFM_SERVICE = "WFM_SERVICE"
    PARCEL_SERVICE_NAME = "PARCEL_SERVICE_NAME"
    NOM_TABLA_PROPS_EXTENDIDAS = "NOM_TABLA_PROPS_EXTENDIDAS"
    CON_INFO_CREAR_JOB_EJECUTOR = "CON_INFO_CREAR_JOB_EJECUTOR"
    dictParams = None
    stepDevolucionRT = "V2.0_CON_AnalisisTecnico"
    notificationRT = "V2.0_CON_RechazadoTotalAV"
    notificationRP = "V2.0_CON_RechazoParcialAV"
    jobStatus = 'Conservacion Ejecucion [EnEjecucion]'
    arcpy.env.overwriteOutput = True
    job = None
    utilsSGC = None
    #gis = GIS("pro")
    gis = GIS(profile='sgc20')

    def __init__(self):
        """Define la herramienta
        - El nombre de la herramienta es el nombre de la clase."""
        self.label = "Devolucion total del flujo"
        self.description = "Devolucion total del flujo"
        utilsSGC = None

    
    def execute(self, parameters, messages):
        """El c\xf3digo fuente de la herramienta."""
        self.utilsSGC = Utils()
        try:
            jobId = parameters[0]
            numeroTramite = parameters[1]
            tipoNotificacion = parameters[2]
            resultado = self.main(jobId, tipoNotificacion, numeroTramite)
            parameters[3] = resultado
            arcpy.AddMessage("Resultado: {}".format(parameters[3]))
        except Exception as e:
            parameters[3] = None
            arcpy.AddError("Ha ocurrido un error: {}".format(e))        
        return


    def main(self, jobId, tipoNotificacion, numeroTramite):
        """Ejecuta el flujo de trabajo"""
        arcpy.AddMessage("Inicio del proceso " + tipoNotificacion)
        self.dictParams = self.utilsSGC.ObtenerParametros()
        result = None
        if tipoNotificacion == "Aprobado":
            result = 1
        if tipoNotificacion == "Rechazo Parcial":
            self.cambiarEstadoJob(jobId)
            self.utilsSGC.sendNotification(jobId, self.notificationRP)
            result = 0
        elif tipoNotificacion == "Rechazo Total":
            versionTramite = self.getVersionName(numeroTramite)
            self.eliminarVersion(self.dictParams[self.PARCEL_SERVICE_NAME], versionTramite)            
            self.crearVersion(versionTramite, self.dictParams, jobId)
            self.cambiarEstadoJob(jobId)
            self.utilsSGC.sendNotification(jobId, self.notificationRT)
            # self.devolverStep(jobId)
            result = 2
        asignacion = self.asignarEjecutor()
        if asignacion["success"] is False:
            raise Exception("Error actualizando funcionario asignado")
        return result


    def devolverStep(self, jobId):
        """codigo para devolver step al principio"""
        arcpy.AddMessage("Devolviendo step")        
        wfm_service = self.dictParams[self.WFM_SERVICE]
        idStepNoCampo = self.getWfmStepID(wfm_service, jobId, self.stepDevolucionRT)
        if idStepNoCampo:
            self.setStepWFM(wfm_service, jobId, idStepNoCampo)
            arcpy.AddMessage("Step devuelto a Preparar Ambiente Movil")
        else:
            arcpy.AddError("Id del step 'Preparar Ambiente Movil:{}' no encontrado, no es posible devolver el paso".format(self.stepDevolucionRT))
            raise Exception("Id del step 'Preparar Ambiente Movil:{}' no encontrado, no es posible devolver el paso".format(self.stepDevolucionRT))


    def cambiarEstadoJob(self, jobId):
        conn = arcpy.wmx.Connect()
        job = conn.getJob(int(jobId))
        arcpy.AddMessage("Cambiando estado de Job a Ejecucion")
        job.status = self.jobStatus
        job.save()

    def asignarEjecutor(self, wfm_service, jobId):
        WfmExtendedProperties = self.utilsSGC.getWfmPropiedadesExtendidas(wfm_service, jobId, self.dictParams[self.NOM_TABLA_PROPS_EXTENDIDAS])
        assignedTo = WfmExtendedProperties["ID_EJECUTOR"]
        if assignedTo is None or len(assignedTo.strip()) == 0:
            raise Exception("Error obteniendo ejecutor en " + self.dictParams[self.NOM_TABLA_PROPS_EXTENDIDAS])
        parametros = {
            "user": self.dictParams[self.CON_INFO_CREAR_JOB_EJECUTOR]["owner"],
            "assignedtype": 1,
            "assignedTo": assignedTo,
            "f": "json",
            "token": ""
        }
        thetoken = self.gis._con.token
        search_result = self.gis.content.get(wfm_service)
        if search_result:
            url_wfm_job_operation = search_result.url+"/jobs/{jobId}/{operation_job}".format(jobId=jobId, operation_job="update")
            parametros["token"] = thetoken
            r = requests.post(url=url_wfm_job_operation, data=parametros)
            resultjson = json.loads(r.text)
            arcpy.AddMessage("Respuesta operacion job:{}".format(resultjson))
            return resultjson

    def eliminarVersion(self, parcel_service, versionName):
        """Eliminar la versionTramite
        Parametros:
        - parcel_service: ID del servicio parcel
        - versionName: nombre de version de edicion que se conciliaran con la version de destino seleccionada
        """
        try:
            arcpy.AddMessage("Eliminando la version {}....".format(versionName))
            thetoken = self.gis._con.token
            search_result = self.gis.content.search('{}'.format(parcel_service),'Feature Layer')
            if search_result:
                url_versions = search_result[0].url.replace("FeatureServer","VersionManagementServer") + "/delete"
                parametros = {
                    "versionName": versionName,
                    "f":"json",
                    "token":thetoken
                }
                r = requests.post(url=url_versions, data=parametros)            
                resultjson = json.loads(r.text)  
                if resultjson["success"]==True:                      
                    return True
                else:
                    return resultjson["error"]["message"]            
        except Exception as ex:
            arcpy.AddMessage("Error eliminando la version: {}".format(str(ex)))     
            return None


    def crearVersion(self, versionName, dictParams, jobId):
        arcpy.AddMessage("Creacion de la version")        
        try:
            parentVersion = dictParams[self.PARENT]
            versionYaExiste = False
            parcel_Service = dictParams[self.PARCEL_SERVICE_NAME]
            listado_versiones = self.listVersions(parcel_Service)
            if listado_versiones:
                for version in listado_versiones:
                    if version["versionName"].find(versionName)!=-1:
                        versionYaExiste = True
                        break
                if versionYaExiste is False:
                    arcpy.AddMessage("La versión no existe. Creando version {}".format(versionName))
                    dict_create = {
                        "user": dictParams[self.CON_INFO_CREAR_JOB_EJECUTOR]["owner"],
                        "name": versionName,
                        "parent": parentVersion,
                        "f":"json",
                        "token": ""
                    }
                    estadocreacionversion = self.crearVersionwmx(wfm_service=dictParams[self.WFM_SERVICE], parametros=dict_create, jobId=jobId)
                
                    if estadocreacionversion==True:
                        arcpy.AddMessage("Version Creada correctamente")                    
                        return True
                    else:
                        arcpy.AddMessage("No se pudo crear la version")                    
                        return False
                else:
                    arcpy.AddMessage("Usando Version Creada Previamente")
                    return True
            else:
                arcpy.AddMessage("No se pudieron listar las versiones")
        except Exception as e:
            arcpy.AddMessage(str(e))
            return False


    def crearVersionwmx(self, wfm_service, parametros, jobId):
        try:    
            arcpy.AddMessage("Creando Version JOB WFM ....")
            resultado = None
            thetoken = self.gis._con.token
            search_result = self.gis.content.search('{}'.format(wfm_service),'Workflow Manager')
            if search_result:
                url_wfm_service_cv = search_result[0].url+"/jobs/{jobId}/createVersion".format(jobId=jobId)
                arcpy.AddMessage(url_wfm_service_cv)
                parametros["token"] = thetoken
                r = requests.post(url=url_wfm_service_cv, data=parametros)
                arcpy.AddMessage("Respuesta creacion version:{}".format(r.text))
                resultjson = json.loads(r.text)
                if resultjson["versionName"]:
                    return True
                else:
                    return False
                resultado = resultjson         
            else:
                arcpy.AddMessage("No se encuentra el servicio {}".format(wfm_service))
            return resultado
        except Exception as ex:
            arcpy.AddMessage("Error creando la version del JOB WFM: {}".format(str(ex)))     
            return None


        def setStepWFM(self, wfm_service, jobId, step_id):
            try:
                arcpy.AddMessage("Set step WFM ....")
                user=self.gis.users.me.username
                resultado = None
                thetoken = self.gis._con.token
                search_result = self.gis.content.search('{}'.format(wfm_service),'Workflow Manager')
                if search_result:
                    url_wfm_service_dep = search_result[0].url+"/jobs/{jobId}/workflow/steps/step/{step}/setAsCurrent".format(jobId=jobId, step=step_id)
                    parametros={
                        "user": user,
                        "f": "json",
                        "token": thetoken
                    }
                    r = requests.post(url=url_wfm_service_dep, data=parametros)
                    resultjson = json.loads(r.text)
                    arcpy.AddMessage("Respuesta set step:{}".format(resultjson))
                    resultado = resultjson         
                else:
                    arcpy.AddMessage("No se encuentra el servicio {}".format(wfm_service))
                return resultado
            except Exception as ex:
                arcpy.AddMessage("Error set step WFM {}".format(str(ex)))     
                return None


        def getWfmStepID(self, wfm_service, jobId, nombre_paso):
            try:
                arcpy.AddMessage("Get WFM Step ID ....")
                stepID = None
                thetoken = self.gis._con.token
                search_result = self.gis.content.search('{}'.format(wfm_service),'Workflow Manager')
                if search_result:
                    url_wfm_service_steps = search_result[0].url+"/jobs/{jobId}/workflow/steps".format(jobId=jobId)            
                    parametros = {
                        "f": "json",
                        "token": thetoken
                    }
                    r = requests.post(url=url_wfm_service_steps, data=parametros)
                    resultjson = json.loads(r.text)
                    for step in resultjson["steps"]:
                        if step["stepType"]["name"] == nombre_paso:
                            stepID = int(step["id"])
                            arcpy.AddMessage("Respuesta Get WFM Step ID job {}:{}".format(jobId,stepID))
                            break
                else:
                    arcpy.AddMessage("No se encuentra el servicio {}".format(wfm_service))
                return stepID
            except Exception as ex:
                arcpy.AddMessage("Error get WFM Step ID: {}".format(str(ex)))     
                return None

    
    def listVersions(self, parcel_service):
        try:
            print("Obteniendo Versiones....")
            listaversiones = None
            thetoken = self.gis._con.token
            search_result = self.gis.content.search('{}'.format(parcel_service),'Feature Layer')
            if search_result:
                url_versions = search_result[0].url.replace("FeatureServer","VersionManagementServer") + "/versions"
                parametros = {
                    "f":"json",
                    "token":thetoken
                }
                r = requests.post(url=url_versions, data=parametros)
                resultjson = json.loads(r.text)
                listaversiones = resultjson["versions"]
            return listaversiones
        except Exception as ex:
            print("Error obteniendo las versiones : {}".format(str(ex)))     
            return None

    def getVersionName(self, numeroTramite):
        """Retorna el nombre de la version del tramite
        Parametros:
        - numeroTramite: Numero del tramite
        """
        self.dictParams[self.CON_INFO_CREAR_JOB_EJECUTOR] = json.loads(self.dictParams[self.CON_INFO_CREAR_JOB_EJECUTOR])
        nameVersionTramite = "{}.CON_{}".format(self.dictParams[self.CON_INFO_CREAR_JOB_EJECUTOR]["owner"], numeroTramite)
        return nameVersionTramite

In [ ]:
class Utils(object):

    def __init__(self):
        super(Utils, self).__init__()

            #ACA SE USA EL NOMBRE DE LA CREDENCIAL ALMACENADA EN EL OS
  search_result = gis.content.search('configuracion','Feature Layer')                        

    def ObtenerParametros(self,where_codigo="1=1"):
        """Obtener parametros configurados en la tabla de configuracion del sistema
        Parametros:    
        - {where_codigo}: codigos para hacer la consulta solamente por esos codigos (Ej: where_codigo='CODIGO1','CODIGO2','CODIGO3')
        ""            arcpy.AddMessage("Obteniendo parámetros....")
            gis = GIS("pro")
          "    
        try:        
            config_item = search_result[0]            
            tabla_conf = config_item.tables[0]
            query_result = tabla_conf.query(where=where_codigo, out_fields='*')
            listado = {}
            for feature in query_result.features:    
                opcion = feature.attributes["opcion"]
                valor = feature.attributes["valor"]    
                listado[str(opcion)]=valor
            #arcpy.AddMessage(listado)     
            return listado
        except Exception as ex:
            arcpy.AddError("Error obteniendo los parametros : {}".format(str(ex)))     
            return None

    def sendNotification(self, pJobid, pNotificacion):
        conn = arcpy.wmx.Connect()
        jobId = int(pJobid)
        job = conn.getJob(jobId)
        job.sendNotification(notification_type=pNotificacion)
        arcpy.AddMessage("Se envi\xf3 exitosamente la notificaci\xf3n a su correo: {0}\n".format(pNotificacion))
        return

    def EliminarArchivosDeDirectorio(self, filtroArchivos):
        """Elimina los archivos con un patrón de ruta
        Parámetros:
        - filtroArchivos: Patrón de búsqueda"""
        filelist = glob.glob(filtroArchivos)
        for file in filelist:
            os.remove(file)
    
        
    def incializarLog(self, jobId):
        '''
        metodo para inicializar el log
        '''
        try:
            now = datetime.now()
            dt_string = now.strftime("%d%m%Y_%H%M%S")
            rootPath = "C:\\Users"
            user = getuser()
            fullpath = os.path.join(rootPath, user, "Documents\\sgc\\tools\\tmp\\")
            name = "ConservPro{}_{}.log".format(jobId,dt_string)
            finalpath = fullpath + name
            if not os.path.exists(fullpath):
                os.makedirs(fullpath, 0o777)
            open(finalpath, "w+")

            logger = logging.getLogger(__name__)
            logger.setLevel(logging.INFO)

            formatter = logging.Formatter('%(asctime)s:%(name)s:%(levelname)s:%(message)s')
            # primer handler para informacion
            file_handler1 = logging.FileHandler(r"" + fullpath + name)
            file_handler1.setLevel(logging.INFO)
            file_handler1.setFormatter(formatter)
            # handlers.append(file_handler1)
            logger.addHandler(file_handler1)
            return [logger, file_handler1, finalpath, fullpath]

        except Exception as error:
            arcpy.AddMessage("Log para el jobId:{} no pudo ser creado".format(jobId))
            arcpy.AddWarning("El log no pudo ser creado o ocurrio un error{}".format(error))

    def cerrarLog(self, handlers):
        for handler in handlers:
            handler.close()

    def generarAleatoreo(self):
        aleatoreo = random.randint(1,100000)
        return aleatoreo

In [ ]:
thetool = ToolDevolucionFlujo()
versionName = "adminsgcv2.CON_08001-2021-TD001"
versionYaExiste = False
ps = parcel_service
listado_versiones = thetool.listVersions(ps)
if listado_versiones:
    for version in listado_versiones:
        if version["versionName"].find(versionName)!=-1:
            versionYaExiste = True
            print("La versión  breakexiste. No Creando version {}".format(versionName))
           
    if versionYaExiste is False:
        print("La versión no existe. Creando version {}".format(versionName))

In [4]:
def reconciliar_versionTramite(parcel_service, versionTramite, postear=True):
    from arcgis.features import FeatureLayerCollection
    gis = GIS(profile=profile_storaged)
    search_result = gis.content.get(parcel_service)
    if search_result:
        p_flc = FeatureLayerCollection.fromitem(search_result)
        version_manager = p_flc.versions
        try:
            bloqueos = version_manager.locks if version_manager.locks else None
            if bloqueos:
                for version_locked in bloqueos:
                    if version_locked.properties.versionName == versionTramite:
                        print("Version bloqueada")
                        desbloqueada = version_manager.purge(version=versionTramite)
                        if desbloqueada is False:
                            raise Exception("No se pudo desbloquear la version!.")
        except Exception as e:
            print("Error revisando bloqueos version: {}".format(e))
        
        try:
            version = version_manager.get(version=versionTramite, mode="edit")
            print(version)
            stop = version.stop_editing()
            print(stop)
            # resultado_concil = version.reconcile()
            # print("Resultado reconciliacion: {}".format(resultado_concil))
            # if resultado_concil and postear:
            #     resultado_post = version.post()
            #     arcpy.AddMessage("Resultado posteo: {}".format(resultado_post))
            #     version_borrada = version.delete()
            #     if version_borrada is False:
            #         arcpy.AddWarning("No fue posible borrar la version")
        except Exception as e2:
            # asunto = "Error ejecutando reconciliacion/posteo, tramite {}".format(versionTramite)
            # mensaje = "{}".format(e2)
            # enviar_correo(asunto, mensaje)
            print("Error reconciliacion/posteo: {}".format(e2))

In [6]:
versionTramite = "adminsgcv2.CON_08001-2021-TD001"
reconciliar_versionTramite(parcel_service, versionTramite, True)
#sgccat20lib.reconciliar_versionTramite(parcel_service, versionTramite, True)

<Version adminsgcv2.CON_08001-2021-TD001 @ {4ABDEA1D-8B1A-4A5A-A672-0E241BF930FD}>
True


In [24]:
#import syssys.executable 'D:\\ProgramFiles\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3-clone\\pythonw.exe'
#from arcgis.gis import GIS
from arcgis.workflow import WorkflowManager

ModuleNotFoundError: No module named 'arcgis.workflow'

In [6]:
versionTramite = "adminsgcv2.CON_08001-2021-Com63a"
sgccat20lib.eliminarVersion(parcel_service, versionTramite)
print("BORRADA la version "+versionTramite)

BORRADA la version adminsgcv2.CON_08001-2021-Com63a
